In [20]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn

In [21]:
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder

In [22]:
with open('intents.json') as file:
    data = json.load(file)

In [23]:
training_sentences =[]
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [24]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels =lbl_encoder.transform(training_labels)

In [25]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 32)                544       
                                                                 
 dense_6 (Dense)             (None, 48)                1584      
                                                      

In [27]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs= epochs)

Epoch 1/500
2/2 [==============================] - 0s 3ms/step - loss: 2.0795 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0788 - accuracy: 0.2121
Epoch 3/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0786 - accuracy: 0.1212
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0781 - accuracy: 0.3030
Epoch 5/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0778 - accuracy: 0.2121
Epoch 6/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0774 - accuracy: 0.2121
Epoch 7/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0769 - accuracy: 0.2121
Epoch 8/500
2/2 [==============================] - 0s 1ms/step - loss: 2.0763 - accuracy: 0.2121
Epoch 9/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0758 - accuracy: 0.2121
Epoch 10/500
2/2 [==============================] - 0s 2ms/step - loss: 2.0755 - accuracy: 0.2121
Epoch 11/500
2/2 [===========

In [28]:
model.save("chat_model")

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [29]:
import pickle

with open ('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('labelEncoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)